In [1]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import logging
tf.get_logger().setLevel(logging.ERROR)
import sys
sys.path.append('code/')
import net
import json
import os
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime as dt
import shutil
import glob
%matplotlib inline

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
path = 'AOI_5_Khartoum_Train/tfrecords/'
training_records = glob.glob(os.path.join(path, '*.tfrecords'))

In [3]:
training_records[:5]

['AOI_5_Khartoum_Train/tfrecords/aoi5_khartoum_train_0720.tfrecords',
 'AOI_5_Khartoum_Train/tfrecords/aoi5_khartoum_train_0757.tfrecords',
 'AOI_5_Khartoum_Train/tfrecords/aoi5_khartoum_train_1283.tfrecords',
 'AOI_5_Khartoum_Train/tfrecords/aoi5_khartoum_train_1500.tfrecords',
 'AOI_5_Khartoum_Train/tfrecords/aoi5_khartoum_train_0317.tfrecords']

In [4]:
training_samples = net.get_from_records(
    training_records, # A list of tfrecord filenames
    batch_size=1,
    buffer=1015,
    shuffle=True,
)

iterator = tf.data.Iterator.from_structure(
    training_samples.output_types,
    training_samples.output_shapes
)
name, image, label = iterator.get_next()

In [5]:
model = net.inference(tf.cast(image, tf.float32))
batch_labels = net.batch_expand_labels(label, [0, 1])
l = net.loss(model, batch_labels)
train_op = net.train(l, 1e-5) # Learning rate

prediction = tf.argmax(model, axis=3, name="prediction", output_type=tf.int32)

def pixel_accuracy(labels, pred):
    ac = tf.divide(
        tf.reduce_sum(tf.cast(tf.equal(pred, labels), tf.float32)),
        tf.cast(tf.size(labels), tf.float32)
    )
    
    return ac

accuracy = pixel_accuracy(tf.cast(label, tf.int32), prediction)

In [6]:
epochs = 10
saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())# Initialize the model parameters
    for epoch in range(epochs):
        sess.run(iterator.make_initializer(training_samples))
        track_loss = []
        start = dt.now()
        while True:
            try:
                _, loss = sess.run([train_op, l]) 
                track_loss.append(loss)
            except tf.errors.OutOfRangeError:
                break
        mean_loss = np.array(track_loss).mean()
        time_taken = (dt.now() - start).seconds
        print('Epoch: {}, loss: {}, time: {}s'.format(epoch, mean_loss, time_taken))
    
    # Save the trained model
    save_path = saver.save(sess, "saved_models/fcn8/model.ckpt")
    print('Model saved to {}'.format(save_path))

Epoch: 0, loss: 0.3058159351348877, time: 591s
Epoch: 1, loss: 0.24635988473892212, time: 585s
Epoch: 2, loss: 0.22117702662944794, time: 585s
Epoch: 3, loss: 0.21011193096637726, time: 585s
Epoch: 4, loss: 0.20025096833705902, time: 585s
Epoch: 5, loss: 0.19241447746753693, time: 585s
Epoch: 6, loss: 0.1891436129808426, time: 585s
Epoch: 7, loss: 0.18671105802059174, time: 585s
Epoch: 8, loss: 0.1816474199295044, time: 585s
Epoch: 9, loss: 0.1783839762210846, time: 585s
Model saved to saved_models/fcn8/model.ckpt


In [8]:
!ls saved_models/fcn8

checkpoint  model.ckpt.data-00000-of-00001  model.ckpt.index  model.ckpt.meta
